In [8]:
import numpy as np
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
from plotly.offline import plot

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

from utils import save_plotly_figure_as_html
from ex_01_read_data import get_welding_data



In [9]:
data_path = Path("data/Welding/data.csv")

## Exercise 2.1: Dataset Statistics Analysis

In this exercise, we'll calculate and present comprehensive descriptive statistics for the welding dataset. This statistical overview will help us understand the fundamental characteristics of our data before visualization or modeling.

1. Calculate the class distribution to understand data balance
2. Count samples per experiment to assess data volume across experimental conditions
3. Determine min/max values for voltage and current to understand the range of measurements
4. Calculate mean and standard deviation to assess central tendency and variability
5. Find median values to identify central points unaffected by outliers

In [10]:
data, labels, exp_ids = get_welding_data(data_path)

In [11]:
# 1. Klassenverteilung (je Sample ein Label)
class_counts = pd.Series(labels).value_counts().sort_index()
print("1) Klassenverteilung:")
print(class_counts)

# 2. Anzahl Samples pro Experiment
exp_counts = pd.Series(exp_ids).value_counts().sort_index()
print("\n2) Samples pro Experiment:")
print(exp_counts)

# 3. Min/Max-Werte über alle Messpunkte hinweg
#    Wir „flatten“ Samples und Timesteps, getrennt für current und voltage
curr_values = data[:, :, 0].ravel()
volt_values = data[:, :, 1].ravel()

print(f"\n3) Spannung: min = {volt_values.min():.3f}, max = {volt_values.max():.3f}")
print(f"   Strom:     min = {curr_values.min():.3f}, max = {curr_values.max():.3f}")

# 4. Mittelwert & Standardabweichung
print(f"\n4) Spannung: mean = {volt_values.mean():.3f}, std = {volt_values.std(ddof=1):.3f}")
print(f"   Strom:     mean = {curr_values.mean():.3f}, std = {curr_values.std(ddof=1):.3f}")

# 5. Medianwerte
print(f"\n5) Spannung: median = {np.median(volt_values):.3f}")
print(f"   Strom:     median = {np.median(curr_values):.3f}")


1) Klassenverteilung:
0    90746
1    67219
Name: count, dtype: int64

2) Samples pro Experiment:
1    35521
2    17913
3    46768
4    57763
Name: count, dtype: int64

3) Spannung: min = 0.074, max = 101.485
   Strom:     min = -1.181, max = 614.154

4) Spannung: mean = 22.037, std = 6.025
   Strom:     mean = 173.456, std = 180.841

5) Spannung: median = 20.841
   Strom:     median = 68.588


## Exercise 2.2: Current and Voltage Distribution Visualization

In this exercise, we'll create interactive boxplots to visualize and compare the distributions of voltage and current measurements in the welding data. Boxplots will help us identify central tendencies, spread, and potential outliers in our measurements.

1. Create side-by-side boxplots for voltage and current using Plotly
2. Display key statistics (median, quartiles, etc.) in a visual format in the plot
3. Enable interactive exploration of the distributions
4. Save the visualization for future reference

In [12]:
data, labels, exp_ids = get_welding_data(data_path, n_samples=10_000)

pio.renderers.default = "png"

In [ ]:
# --- Pfade & Ordner anlegen ---
data_path     = Path("data/Welding/data.csv")
plot_folder   = Path("plots/ex_02")
docs_folder   = Path("docs")
html_filename = "voltage_current_distribution.html"

plot_folder.mkdir(parents=True, exist_ok=True)
docs_folder.mkdir(parents=True, exist_ok=True)

html_save_plot = plot_folder / html_filename
html_save_docs = docs_folder  / html_filename
png_save_plot  = plot_folder / "voltage_current_distribution.png"

# --- Daten laden (korrekt unpacken!) ---
data, labels, exp_ids = get_welding_data(data_path, n_samples=10_000)
# data.shape == (n_samples, timesteps, 2)
# feature-index 0 = current, 1 = voltage

# --- Variante 2: Mittelwerte pro Zyklus berechnen ---
mean_currents = data[:, :, 0].mean(axis=1)
mean_voltages = data[:, :, 1].mean(axis=1)

# --- DataFrame & langes Format ---
df = pd.DataFrame({
    "Mean Current (A)": mean_currents,
    "Mean Voltage (V)": mean_voltages
})
df_long = df.melt(var_name="Measurement", value_name="Mean Value")

# --- Kennzahlen für Titel ---
med_curr   = np.median(mean_currents)
q1_curr, q3_curr = np.percentile(mean_currents, [25, 75])
med_volt   = np.median(mean_voltages)
q1_volt, q3_volt = np.percentile(mean_voltages, [25, 75])

# --- Interaktiven Boxplot erzeugen ---
fig = px.box(
    df_long,
    x="Measurement",
    y="Mean Value",
    points="all",
    title=(
        "Verteilung der Stichproben-Mittelwerte<br>"
        f"Current: median={med_curr:.2f}, Q1={q1_curr:.2f}, Q3={q3_curr:.2f}<br>"
        f"Voltage: median={med_volt:.2f}, Q1={q1_volt:.2f}, Q3={q3_volt:.2f}"
    ),
    labels={"Mean Value": "Durchschnittlicher Messwert", "Measurement": "Messgröße"},
    template="plotly_white"
)
fig.update_layout(width=800, height=500, margin=dict(l=40, r=40, t=100, b=40), showlegend=False)

# --- Speichern ---
save_plotly_figure_as_html(fig, html_save_plot)          # local plots/
fig.write_html(str(html_save_docs), include_plotlyjs="cdn")  # docs/ für GitHub Pages
fig.write_image(str(png_save_plot))                         # optional PNG

# --- Inline-Anzeige ---
fig.show(renderer="png")

print(f"• plots-HTML: {html_save_plot}")
print(f"• docs-HTML:  {html_save_docs}")
print(f"• PNG:        {png_save_plot}")

IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

## Exercise 2.3: Time-Series Sequence Visualization

In this exercise, we'll implement functions to visualize the time-series patterns of welding cycles. These visualizations will help us understand the temporal relationships between voltage and current during the welding process and identify patterns associated with quality outcomes.

1. Create dual-axis plots showing voltage and current over time (10 welding cycles -> sequence_length=10) using matplotlib
2. Implement clear legends and labels for data interpretation
3. Enable saving of visualizations for reporting and comparison

In [ ]:

data, labels, exp_ids = get_welding_data(data_path, n_samples=100, return_sequences=True, sequence_length=10)
save_path = plot_path / "welding_sample.png"    

NameError: name 'plot_path' is not defined

## Exercise 2.4: Interactive Time-Series Visualization with Plotly

In this exercise, we'll create enhanced interactive visualizations of welding samples using Plotly. These interactive plots will provide more advanced exploration capabilities for analyzing the time-series patterns.

1. Create interactive plots with dual y-axes for voltage and current
2. Implement time-based range sliders for detailed exploration
3. Add unified tooltips for precise data reading
4. Display quality metrics in the plot title
5. Save interactive visualizations as HTML for sharing

In [ ]:
def create_plotly_plot(data: np.ndarray, labels: np.ndarray, exp_ids: np.ndarray = None) -> go.Figure:
    """
    Create an interactive Plotly visualization of a random welding sample.

    Args:
        data (np.ndarray): Array containing voltage and current data
        labels (np.ndarray): Array containing class labels
        exp_ids (np.ndarray, optional): Array containing experiment IDs. Defaults to None.

    Returns:
        plotly.graph_objects.Figure: Interactive Plotly figure object
    """
    pass


fig = create_plotly_plot(data, labels, exp_ids)
save_plotly_figure_as_html(fig, plot_path / "welding_samples")
fig.show()

AttributeError: 'NoneType' object has no attribute 'write_html'

## Exercise 2.5: Multiple Sample Comparison

In this exercise, we'll generate and compare visualizations from multiple random welding samples. This comparison will help us identify common patterns and variations across different welding cycles.
 

1. Generate multiple random sample visualizations using matplotlib of single welding cycles
2. Create dual-axis plots showing voltage and current over time
3. Implement clear legends and labels for data interpretation
4. Save each visualization for comparison

In [ ]:
data, labels, exp_ids = get_welding_data(data_path, n_samples=1_000)

In [ ]:
for i in range(5):
    plot_random_sample(data, labels, save_path=plot_path / f"welding_sample_{i}.png")
    